In [ ]:
!pip install azure-cli
!pip install pydicom azure.identity
!pip install requests_toolbelt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 29.8 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 88 kB 9.8 MB/s 
     |████████████████████████████████| 1.2 MB 71.2 MB/s 
     |████████████████████████████████| 106 kB 48.1 MB/s 
     |████████████████████████████████| 47 kB 5.9 MB/s 
     |████████████████████████████████| 58 kB 6.8 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 117 kB 53.3 MB/s 
     |████████████████████████████████| 5.5 MB 36.8 MB/s 
     |████████████████████████████████| 93 kB 1.7 MB/s 
     |████████████████████████████████| 7.8 MB 51.9 MB/s 
     |████████████████████████████████| 484 kB 61.8 MB/s 
     |████████████████████████████████| 87 kB 7.6 MB/s 
     |████████████████████████████████| 291 kB 62.4 MB/s 
     |████████████████████████████████| 94 kB

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 27.0 MB/s 
     |████████████████████████████████| 136 kB 44.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.8 MB/s 


In [ ]:
import requests
import pydicom
from pathlib import Path
import matplotlib.pylab as plt
from azure.identity import DefaultAzureCredential
from urllib3.filepost import encode_multipart_formdata, choose_boundary
import requests_toolbelt as tb
from io import BytesIO
import pandas as pd
from PIL import Image

In [ ]:
base_url = f"https://mammogram-mammogram.dicom.azurehealthcareapis.com/v1"

In [ ]:
mass_train=pd.read_csv('/content/drive/MyDrive/mass_case_description_train_set.csv')
mass_train['studyid']=mass_train['image file path'].apply(lambda x: x.split('/')[1])
mass_train['seriesid']=mass_train['image file path'].apply(lambda x: x.split('/')[2])
feature_label_train_mass=mass_train[['studyid','pathology']]

In [ ]:
def azlogin():
  !az login

In [ ]:
def get_cred_and_check_auth():
  

  credential = DefaultAzureCredential()

  #print(credential.credentials) # this can be used to find the index of the AzureCliCredential
  token = credential.credentials[4].get_token('https://dicom.healthcareapis.azure.com')
  bearer_token = f'Bearer {token.token}'
  def encode_multipart_related(fields, boundary=None):
      if boundary is None:
          boundary = choose_boundary()

      body, _ = encode_multipart_formdata(fields, boundary)
      content_type = str('multipart/related; boundary=%s' % boundary)

      return body, content_type
    
  client = requests.session()

  headers = {"Authorization":bearer_token}
  url= f'{base_url}/changefeed'

  response = client.get(url,headers=headers)
  if (response.status_code != 200):
      print('Error! Likely not authenticated!')
  else:
    print("Success")
  return bearer_token,client

In [ ]:
azlogin()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FM65YA3CQ to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "e056658c-7855-40e3-9a64-9a8c4fe82c9a",
    "id": "c7518a1a-6651-45cb-8726-8fd7cf93e0cb",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure for Students",
    "state": "Enabled",
    "tenantId": "e056658c-7855-40e3-9a64-9a8c4fe82c9a",
    "user": {
      "name": "spp26041999@gmail.com",
      "type": "user"
    }
  }
]


In [20]:
bearer_token, client = get_cred_and_check_auth()

Success


In [21]:
def image_fetch(studyid_idx):
  url = f'{base_url}/studies/{feature_label_train_mass.studyid[studyid_idx]}'
  headers = {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*', "Authorization":bearer_token}

  response = client.get(url, headers=headers) #, verify=False)
  mpd = tb.MultipartDecoder.from_response(response)

  mpd = tb.MultipartDecoder.from_response(response)
  part=mpd.parts[0]
  ds = pydicom.dcmread(BytesIO(part.content))
  return ds.pixel_array

In [22]:
for i in range(1005,1111):#1318):
  Image.fromarray(image_fetch(i)).save(f"{feature_label_train_mass.studyid[i]}.png")
  print(i)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 396, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/response.py", line 72, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [StartBoundaryNotFoundDefect(), MultipartInvariantViolationDefect()], unparsed data: ''


1005


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 396, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/response.py", line 72, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [StartBoundaryNotFoundDefect(), MultipartInvariantViolationDefect()], unparsed data: ''


1006


NonMultipartContentTypeException: ignored